## Measure contribution of Major merger, minor merger, and smooth accretion for only the 'safe' samples...?. 
Because tree bad link more likely occur at major merger events, I guess the 'safe' samples have less major mergers than the total sample. 

## import analysis.Major_Minor_accretion as mma


Largest halo in the refinement region is not the main cluster.
Is the largest NP halo the main cluster? 
To check it, color halos in NP.

No, max_np galaxy/halo is the main galaxy/halo.
But 'rvir' value is wrong.

and fixed.

In [1]:
import utils.sampling as smp
import matplotlib.pyplot as plt
import tree
import pickle
import tree.halomodule as hmo
import numpy as np
from analysis.misc import load_cat
import scipy.stats
import tree.ctutils as ctu
from analysis.evol_lambda import MainPrg
import draw
import load
import analysis.evol_lambda as evl
import analysis.Major_Minor_accretion as mma
import analysis.misc as amsc
import tree.ctutils as ctu
import utils.match as mtc
# Read a single galaxy evolution catalog.

from MajorMinorAccretion_module import *

verbose=True
# In[4]:
base = './'
cdir = ['catalog/', 'easy/', 'catalog_GM/', "easy_final/"][3]

clusters = ['01605', '07206', \
            '35663', '24954', '49096', \
            '05427', '05420', '29172', \
            '29176', '10002', '36415', \
            '06098', '39990', '36413', \
            '17891', '04466']
# parameters used for lambda_arr clipping.
#ind_upper = 20
#ind_lower = 20
#sig_upper = 2.0
#sig_lower = 2.0

nout_ini = 37 # 62: z = 1.666
nout_fi = 187

minimum_good_snap = 87


In [3]:
def body(clusters,
         dist_gal_scale_in=5,
         dist_gal_scale_out=10,
         dt_before=0.5,
         dt_after=0.5,
         dt_settle=0.5,
         load=False):

    suffix = "_{}_{}_{}_{}_{}".format(dist_gal_scale_in,
    dist_gal_scale_out,dt_before,dt_after,dt_settle)

    if load:
        return pickle.load(open("main_prgs_final_augmented" + suffix + ".pickle", 'rb'))
        
    else:
        mpgs = []
        for cluster in clusters:
            print(cluster)
            wdir = base + cluster + '/'

            # Serialize catalogs. -> Only main galaxies
            # main galaxy list
            alltrees = ctu.load_tree(wdir, is_gal=True)
            ad = alltrees.data
            tn = ad[ad['nout'] == nout_fi]

            cat = load_cat(wdir + cdir + 'catalog' + str(nout_fi) + '.pickle')
            #idx_all = [tn['id'][tn['Orig_halo_id'] == id_final][0] for id_final in cat['id']]
            idx_all = cat['idx'][cat["idx"] > 0].astype(int) # why idx are float???

            mpg_tmp = []
            for i, idx in enumerate(idx_all):
                #print(i, idx)

                mpg_tmp.append(MainPrg(ad, idx))
            #    mpg_tmp =[MainPrg(ad, idx) for idx in idx_all]
            for nout in range(nout_ini, nout_fi + 1):
                cat = load_cat(wdir + cdir + 'catalog' + str(nout) + '.pickle')
                for gal in mpg_tmp:
                    gal.set_data(cat, nout)
                    gal.cluster = int(cluster)
        #        print(nout)
            # get rid of galaxies with too short tree.
            mpg_tmp = [gg for gg in mpg_tmp if sum(gg.data["reff"] > 0) > minimum_good_snap]
            for gal in mpg_tmp:
                gal.fill_missing_data()
                gal.clip_non_detection()
                gal.smoothed_lambda_org = mma.smooth(gal.data["lambda_r"], window_len=15)[:-1]
                gal.smoothed_r = mma.smooth(gal.data["reff"], window_len=15)[:-1]
                gal.smoothed_lambda = mma.smooth(l_at_smoothed_r(gal, npix_per_reff=5), window_len=15)[:-1]
            # Find_merger_epochs needs smoothed_r
            find_merger_epochs(alltrees,
                               idx_all,
                               mpg_tmp,
                               nout_ini=nout_ini,
                               dist_gal_scale_in=dist_gal_scale_in,
                               dist_gal_scale_out=dist_gal_scale_out,
                               mass_ratio='early',
                               verbose=False,
                               do_plot=False,
                               pdf_fname=str(cluster) + "merger_ratio_epoch" + suffix + ".pdf")

            # save for each cluser
            with open(wdir + "main_prgs" + suffix + ".pickle", "wb") as f:
                pickle.dump(mpg_tmp, f)

            while len(mpg_tmp) > 0:
                mpgs.append(mpg_tmp.pop())


        for gal in mpgs:
            # Keep only the largest merger among multiple mergers
            filter_small_mergers(gal.merger)

        with open("main_prgs" + suffix + ".pickle", 'wb') as f:
            mpgs.pop(2)
            pickle.dump(mpgs, f)

        measure_delta(mpgs,
                      dt_before=dt_before,
                      dt_after=dt_after,
                      dt_settle=dt_settle,
                      nout_ini=nout_ini,
                      savefig=False,
                      figname="figs/measure_delta" + suffix)

        Maj_min_acc_ratio(mpgs, major_ratio=4)

        with open("main_prgs_final_augmented" + suffix + ".pickle", 'wb') as f:
            pickle.dump(mpgs, f)

In [ ]:
def kde_den(data, cov=0.25):
    from scipy.stats import gaussian_kde
    density = gaussian_kde(data)
    xs = np.linspace(0,8,200)
    density.covariance_factor = lambda : cov
    density._compute_covariance()
    return density

def draw_kdes(dlM, dlm, dlo, ax
                  , nevents
                  , lw=2):
        dM = kde_den(dlM)
        dm = kde_den(dlm)
        do = kde_den(dlo)

        nM = len(dlM)
        nm = len(dlm)
        no = len(dlo)

        xs=np.linspace(-0.6,0.6,50)
        Mlabel="Major \n" +r"$N_{g}(N_{e})$" + " = {}({})".format(nM, nevents[0])
        mlabel="Minor \n" +r"$N_{g}(N_{e})$" + " = {}({})".format(nm, nevents[1])
        olabel="Rest  \n" +r"$N_{g}(N_{e})$" + " = {}({})".format(no, nevents[2])
        ax.plot(xs, dM(xs)*nM/no, label=Mlabel, lw=lw, color="r")
        ax.plot(xs, dm(xs)*nm/no, label=mlabel, lw=lw, color="g")        
        ax.plot(xs, do(xs), label=olabel, lw=lw, color="b")
        

In [127]:
def kde_sci(mpgs
    ,mstar_cut_hard = 5e9
    ,mcut=1e10
    ,fname="figs/test"
    ,wdir='./'
    ,nbins=21
    ,kde=True
    ,hist=True
    ,shade=True
    ,norm_hist=False
    ,pallette="muted"
    ,ylim=None
    ,per_event=True
    ,per_galaxy=True
    ,detected=True
    ,maj_ratio = 4):
    
    from matplotlib.ticker import NullFormatter

    l_dl_e = []
    l_mr_e = []
    l_mass_e = []

    s_dl_e = []
    s_mr_e = []
    s_mass_e = []

    l_dlt_g=[]
    l_dlo_g=[]
    l_dlM_g=[]
    l_dlm_g=[]
    l_mass_g=[]

    s_dlt_g=[]
    s_dlo_g=[]
    s_dlM_g=[]
    s_dlm_g=[]
    s_mass_g=[]


    M_changed = 0
    m_changed = 0
    no_merger_count = 0
    count = 0
    Maj_small = 0
    for i, gal in enumerate(mpgs):
        mgal = gal.data["mstar"][0]
        if mgal > mstar_cut_hard:
            delta_lambda_tot = np.average(gal.data['lambda_r'][:5]) - np.average(gal.data['lambda_r'][-5:])
            delta_lambda_major = 0
            delta_lambda_minor = 0        

            # Large
            if mgal > mcut:
                if hasattr(gal, "merger"):
                    if gal.merger is not None:
                        l_dl_e.extend(gal.merger.delta_l)
                        l_mr_e.extend(gal.merger.mr)
                        for dl, mr in zip(gal.merger.delta_l, gal.merger.mr):
                            if (mr < maj_ratio) and (dl > -1):
                                delta_lambda_major = delta_lambda_major + dl
                            if (mr > maj_ratio) and (dl > -1):
                                delta_lambda_minor = delta_lambda_minor + dl

                delta_lambda_other = delta_lambda_tot - delta_lambda_major - delta_lambda_minor
                l_dlt_g.append(delta_lambda_tot)
                l_dlo_g.append(delta_lambda_other)
                l_dlM_g.append(delta_lambda_major)
                l_dlm_g.append(delta_lambda_minor)
            # small
            else:
                #s_mass_g.append(mgal)
                if hasattr(gal, "merger"):
                    if gal.merger is not None:
                        s_dl_e.extend(gal.merger.delta_l)
                        s_mr_e.extend(gal.merger.mr)
                        for dl, mr in zip(gal.merger.delta_l, gal.merger.mr):
                            if (mr < maj_ratio) and (dl > -1):
                                delta_lambda_major = delta_lambda_major + dl
                            if (mr > maj_ratio) and (dl > -1):
                                delta_lambda_minor = delta_lambda_minor + dl

                    delta_lambda_other = delta_lambda_tot - delta_lambda_major - delta_lambda_minor
                    s_dlt_g.append(delta_lambda_tot)
                    s_dlo_g.append(delta_lambda_other)
                    s_dlM_g.append(delta_lambda_major)
                    s_dlm_g.append(delta_lambda_minor)

    l_dlt_g = np.array(l_dlt_g)
    l_dlo_g = np.array(l_dlo_g)
    l_dlM_g = np.array(l_dlM_g)
    l_dlm_g = np.array(l_dlm_g)
    #l_mass_g = np.array(l_mass_g)

    s_dlt_g = np.array(s_dlt_g)
    s_dlo_g = np.array(s_dlo_g)
    s_dlM_g = np.array(s_dlM_g)
    s_dlm_g = np.array(s_dlm_g)
    #s_mass_g = np.array(s_mass_g)

    # detected
    l_dlM_g = l_dlM_g [l_dlM_g !=0]
    #l_dlM_M = l_mass_g[l_dlM_g !=0]
    l_dlm_g = l_dlm_g [l_dlm_g !=0]
    #l_dlm_M = l_mass_g[l_dlm_g !=0]
    #l_dlo_M = l_mass_g

    s_dlM_g = s_dlM_g [s_dlM_g !=0]
    #s_dlM_M = s_mass_g[s_dlM_g !=0]
    s_dlm_g = s_dlm_g [s_dlm_g !=0]
    #s_dlm_M = s_mass_g[s_dlm_g !=0]
    #s_dlo_M = s_mass_g


    l_dl_e = np.array(l_dl_e)
    l_mr_e = np.array(l_mr_e)
    #l_mass_e = []

    s_dl_e = np.array(s_dl_e)
    s_mr_e = np.array(s_mr_e)
    #s_mass_e = []

    fig, axs = plt.subplots(3, sharex=True)
    fig.set_size_inches(7,10)
    plt.subplots_adjust(hspace=0.01)

    all_dlM_g = np.concatenate((l_dlM_g,s_dlM_g))
    all_dlm_g = np.concatenate((l_dlm_g,s_dlm_g))
    all_dlo_g = np.concatenate((l_dlo_g,s_dlo_g))

    draw_kdes(all_dlM_g,
              all_dlm_g,
              all_dlo_g,
              axs[0],
              [sum(s_mr_e < maj_ratio) + sum(l_mr_e < maj_ratio),
               sum(s_mr_e > maj_ratio) + sum(l_mr_e > maj_ratio),
               len(all_dlo_g)])

    draw_kdes(l_dlM_g,
              l_dlm_g,
              l_dlo_g,
              axs[1],
              [sum(l_mr_e < maj_ratio),
               sum(l_mr_e > maj_ratio),
               len(l_dlo_g)])

    draw_kdes(s_dlM_g,
              s_dlm_g,
              s_dlo_g,
              axs[2],
              [sum(s_mr_e < maj_ratio),
               sum(s_mr_e > maj_ratio),
               len(s_dlo_g)])

    axs[0].set_xlim([-0.6,0.6])
    for ax in axs:
        ax.xaxis.grid()
        ax.legend(fontsize=11)
        ax.yaxis.set_major_formatter(NullFormatter())
        ax.set_ylabel("relative probability")

    axs[2].set_xlabel(r"$\Delta \lambda_{R_{eff}}$", fontsize=14)
    axs[2].tick_params(labelsize=12)
    axs[2].set_xlim([-0.7,0.6])
    #axs[0].legend(fontsize=12)

    axs[0].text(0.05, 0.87, "All",transform=axs[0].transAxes, fontsize=13)
    axs[1].text(0.05, 0.87, r"$log_{10}M_{\star} > $ " +"{:.1f}".format(np.log10(mcut))
                , fontsize=13
                , transform=axs[1].transAxes)
    axs[2].text(0.05, 0.87, r"$log_{10}M_{\star} < $ " +"{:.1f}".format(np.log10(mcut))
                , fontsize=13
                , transform=axs[2].transAxes)


    plt.savefig(fname + "{:.1f}.png".format(np.log10(mcut)), dpi=200, bbox_inches="tight")
    plt.savefig(fname + "{:.1f}.pdf".format(np.log10(mcut)), bbox_inches='tight') # eps does NOT support transparency!

    plt.close()

In [5]:
mpgs = body(clusters,
        dist_gal_scale_in=5,
        dist_gal_scale_out=10,
        dt_before=0.5,
        dt_after=0.5,
        dt_settle=0.5,
        load=True)

In [130]:
kde_sci(mpgs,
        mstar_cut_hard = 5e9,
        mcut = 3.3e10,
        hist=False,
        shade=False,
        kde=True,
        norm_hist=False,
        detected=True,
        maj_ratio=4,
        fname="figs/MajMinNon_contribution_fi")